In [1]:
import os 
import sys 
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) 
if project_root not in sys.path: 
    sys.path.insert(0, project_root)

from data.Sampler import Sampler 
from data.TrajectorySet import TrajectorySet
from models.cl_model import mlpCL

import torch

import minari

In [7]:
class RandomSamplingDataset(torch.utils.data.Dataset): 
    def __init__(self, cl_model,  sampler: Sampler, num_states: int):
        """
        Creates a dataset with randomly sampled states from the minari dataset specified by the TrajectorySet class. 
        """
        self.cl_model = cl_model 

        states = torch.tensor(sampler.sample_states(batch_size=num_states), dtype=torch.float32)
        self.z = cl_model(states)

    def __len__(self): 
        return len(self.z) 
    
    def __getitem__(self, index):
        return self.z[index]


In [3]:
T = TrajectorySet(dataset = minari.load_dataset("D4RL/pointmaze/large-v2"))
S = Sampler(T=T) 

model_name = "best_model.ckpt"
pretrained_model_file = os.path.join(project_root+ "/saved_models", model_name) 

if os.path.isfile(pretrained_model_file): 
    print(f"Found pretrained model at {pretrained_model_file}, loading...") 
    cl_model = mlpCL.load_from_checkpoint(pretrained_model_file, map_location=torch.device("cpu"))

Found pretrained model at /Users/ray/Documents/Research Assistancy UofA 2025/Reproduce Paper/contrastive-abstraction-RL/saved_models/best_model.ckpt, loading...


In [11]:
RSD = RandomSamplingDataset(cl_model=cl_model, sampler=S, num_states=50) 

dl = torch.utils.data.DataLoader(dataset=RSD, batch_size=10, shuffle=True)
for i in dl: 
    print(i.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
